In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('../commonlit-evaluate-student-summaries/summaries_train.csv')

In [6]:
embeddings = np.load('../embeddings.npy')

In [7]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from transformers import ElectraTokenizer, ElectraModel
from transformers import RobertaTokenizer, RobertaModel


from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # Import tqdm for the progress bar
from transformers import RobertaTokenizer

2023-08-19 22:54:20.473962: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 22:54:20.512896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 22:54:22.229649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [34]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, train_test_split

In [8]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [9]:
max_seq_length = 512

def tokenize_text(text):
    return tokenizer.encode(text, add_special_tokens=True, max_length=max_seq_length, padding='max_length', truncation=True)

data['text_tokens'] = data['text'].apply(tokenize_text)

In [41]:
data.to_csv('../saved_data/data_roberta_tokens.csv', index=False)

In [10]:
roberta_model = RobertaModel.from_pretrained("roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def generate_roberta_embeddings(text_tokens):
    embeddings_list = []
    with tqdm(total=len(text_tokens)) as pbar:
        with torch.no_grad():
            for tokens in text_tokens:
                outputs = roberta_model(tokens.unsqueeze(0))  # Unsqueeze to add batch dimension
                embeddings = outputs.last_hidden_state[:, 0, :]  # Extract embeddings for [CLS] token
                embeddings_list.append(embeddings)
                pbar.update(1)
        embeddings_tensor = torch.cat(embeddings_list, dim=0)
        return embeddings_tensor

In [12]:
X = data['text_tokens'][:100].tolist()
X = torch.tensor(X)

In [13]:
X_embeddings = generate_roberta_embeddings(X)

100%|██████████| 100/100 [00:10<00:00,  9.24it/s]


In [23]:
X_embeddings.numpy().shape

(100, 768)

In [24]:
np.save("/home/woody/iwso/iwso092h/student_summaries/saved_data/sample_roberta_embeddings.npy", X_embeddings.numpy())

In [25]:
roberta_emb = np.load('../saved_data/roberta_embeddings.npy')

In [27]:
roberta_emb.shape

(7165, 768)

In [30]:
X = roberta_emb
y = data[['wording', 'content']].values

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
regressor = XGBRegressor()
multi_output_regressor = MultiOutputRegressor(regressor)

In [35]:
cv_scores = cross_val_score(multi_output_regressor, X_train, y_train, cv=4, scoring='neg_mean_squared_error')
print("Cross-Validation Scores (Negative Mean Squared Error):")
print(cv_scores)

Cross-Validation Scores (Negative Mean Squared Error):
[-0.47115731 -0.4476232  -0.44923079 -0.44920358]


In [37]:
multi_output_regressor.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))

In [39]:
pred = multi_output_regressor.predict(X_test)

In [45]:
mean_squared_error(pred[:,0], y_test[:,0])

0.5740998314326511

In [46]:
mean_squared_error(pred[:,1], y_test[:,1])

0.3030991392768576